# Fully Connected Auto-Encoder wb Sweeps
### $Time$ $Series$ $3rd$ $Test$

$Vasco$ $Mergulhão$ $-$ $Jan$ $2023$

### Version 1:
 - Applies Weights and Biases Sweeps on a given Sub-Sample.
 - Only uses FC-AE architectures

### ANN Configurations:

- #### Architecture(s)
    - Fully Connected Auto Encoder
        - Small (Input, 200, 200, Enconder)
        - Medium (Input, 300, 300, 300, 300, Encoder)
        - End2End [other papers orig ref] (Input, 500, 500, 2000, Encoder)
    
- #### Hyperparamenters
    - Latent Space Size
    - Batch Size
        - Small test [2 - 32] and Large test [128 - 256]
    - Learning Rate
    - Learning Rate Scheduler
        - Performance Schedulling
    - Activation Functions
        - SELU and Leaky ReLU
    - Initializations
        - LeCun and He (accordingly)
    - Batch Normalization
        - With/Without tests (note: if data is not z-scored, SELU not worth it, downgrade to ELU)
    - Optimizers
        - Nadam and SDG(momentum [0.9], Nesterov)
    - Epochs
        - 100 with Early Stopping
 

---
# 

In [1]:
import os
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, date
import time
import datetime
import scipy

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU, Activation
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow import keras

import wandb
from wandb.keras import WandbCallback

from sklearn.model_selection import train_test_split

---
# Script Variables

In [2]:
# Defines Dataset for the Sweep
dataset_name = 'Kenya_10k_Set_1_w90'

# Uses name to navigate folders
dataset_folder = "_".join(dataset_name.split('_')[:-1]) #Takes out window length section
dataset_location = f'Data/{dataset_folder}/{dataset_name}.csv'

# Zcore Data Decision
zscore_data = True # Set to: [True/False]
zscore_data_done = False # Always set to False. Ensures its not normalized multiple times

# Sweep Names and Tags
Project_Name = f'FC_AE-{dataset_name}'
AE_Model_Name = 'FC_Small' # Options: FC_Small, FC_Medium
Sweep_Config = f'{AE_Model_Name}_v1'
sweep_count = 6
# Model variables
latent_layer_size = 5

In [3]:
def window_col_names(dataset_name, win_prefix = 'd'):
    # retriving window length
    window_len = int(dataset_name.split('_')[-1][1:]) # Gets _wXX part of name, then skips 'w' to get the number.
    # defining window column names
    window_cols = [None]*window_len
    for i  in range(window_len):
        window_cols[i] = f'{win_prefix}' + str(i+1)
        
    return window_cols

---
# Data Imports

In [4]:
Data = pd.read_csv(dataset_location)

In [5]:
Data.head()

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,347,0,2018-01-19,15.422222,14.422222,13.422222,12.422222,11.422222,10.422222,9.422222,...,8.696088,6.709653,6.696088,5.696088,3.730243,3.696088,1.740706,0.744016,-7.000000,-7.000000
1,347,1,2018-04-19,6.475880,4.490544,4.475880,3.475880,2.475880,1.475880,-7.000000,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,6.634468,5.634468,4.634468
2,347,2,2018-07-18,3.634468,2.634468,1.634468,3.634468,2.634468,1.634468,0.634468,...,4.762303,3.762303,2.762303,1.762303,0.762303,5.762303,4.762303,3.762303,2.762303,1.762303
3,347,3,2018-10-16,0.762303,1.762303,0.762303,2.762303,1.762303,1.762303,0.762303,...,3.553345,2.553345,1.553345,0.553345,-7.000000,6.646019,5.646019,4.646019,3.646019,2.646019
4,347,4,2019-01-14,1.646019,0.646019,2.646019,1.646019,0.646019,2.646019,1.646019,...,0.613681,2.613681,1.613681,1.613681,0.613681,1.613681,2.613681,1.613681,1.613681,0.613681


---
# Z-Scoring Data

This is done on a row-by-row basis.<br>
Meaning, each window is normalized to its own Mean and Std.

In [6]:
def Zscore (df_in, all_neg_replace = -1):
    # retriving sets of columns
    window_cols = window_col_names(dataset_name)        
    cols = df_in.columns.to_list()
    other_cols = list(set(cols) - set(window_cols))
    
    # Zscore can't handle constant (flat) inputs
    # Therefore windows with all negative credit output NaNs
    # Instead these are excluded from zcoring and are manually scaled
    # Default scalling is  -7 => -1
    all_neg_replace = float(all_neg_replace)
    all_neg_index = df_in[(df_in[window_cols] == float(-7)).all(axis=1)].index
    positive_index = list(set(df_in.index.to_list()) - set(all_neg_index))
    
    if len(all_neg_index.to_list()) > 0:
        # Aux DF for replacing -7 with all_neg_replace (i.e, -1)
        df_allneg = df_in.iloc[all_neg_index].copy()
        df_allneg.loc[:, window_cols] = all_neg_replace

        # Zscoring only non-constant rows
        df_positive = df_in.iloc[positive_index].copy()    
        df_positive[window_cols] = scipy.stats.zscore(df_positive[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')

        df_zscore = pd.concat([df_positive, df_allneg])
        df_zscore.sort_values(by=['short_ID', 'window_ID'], inplace = True)

        # If indeces match, then the join was successful
        if df_zscore.index.to_list() == df_in.index.to_list():
            return df_zscore
        # Otherwise raise error
        else:
            print('Error Zscoring')
    else:
        df_zscore = pd.DataFrame(columns= cols)
        df_zscore[window_cols] = scipy.stats.zscore(df_in[window_cols],
                                                         axis=1,
                                                         nan_policy = 'omit')
        df_zscore[other_cols] = df_in[other_cols]
        

    return df_zscore

In [7]:
if zscore_data == True and zscore_data_done == False:
    Data = Zscore(Data)
    zscore_data_done = True
    print('Data WAS Zscored')
    
elif zscore_data == True and zscore_data_done == True:
    print('Already WAS Zscored')
    
elif zscore_data == False:
    print('Data NOT Zscored')
    
else:
    print('Error')

Data WAS Zscored


---
# NaN Checks and Policy

In [8]:
def NaN_policy (df, method_fullrows = 'ffill', method_sparserows = 'ffill'):
    # This functions checks and corrects NaNs.
    # It has redundancy built into it to double check for NaNs.
    # And allows for different policies for sparse NaN values and rows full of NaNs.
    
    df = df.copy(deep=True)
    window_cols = window_col_names(dataset_name)
        
    if df[window_cols].isna().any().any():
        print('NaN values detected.')
        
        nan_exist = True
        i = 0   
        while nan_exist:
            
            # Makes sure the policy only iterates with one redundant round.
            if i >= 2:
                print('NaN Policy Failed')
                break            

            #Checking if there are rows with only NaNs
            if df[window_cols].isna().all(axis=1).any():
                n_nan_rows = len(df[df[window_cols].isna().all(axis=1)])
                print(f'There are {n_nan_rows} rows of all NaN values.')

                #Correcting only NaN rows 
                df.fillna(method= method_fullrows, axis = 0, inplace = True)
                if ~ df[window_cols].isna().all(axis=1).any():
                    print(f'Rows of NaN corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')
            
            #Checking if there are rows with any NaNs in them
            if df[window_cols].isna().any(axis=1).any():
                n_nan_rows = len(df[window_cols].isna().any(axis=1))
                print(f'There are {n_nan_rows} rows with some NaN values.')
                
                # Correcting NaN inside rows 
                df.fillna(method= method_sparserows, axis = 1, inplace = True)
                if ~ df[window_cols].isna().any(axis=1).any():
                    print(f'Rows with some NaN were corrected')
                    if ~ df[window_cols].isna().any().any():
                        nan_exist = False
                        print(f'NaNs no longer detected.')

            i += 1
            

    else:
        print('No NaNs detected')
        
    return df

In [9]:
Data = NaN_policy(Data)

NaN values detected.
There are 22 rows of all NaN values.
Rows of NaN corrected
NaNs no longer detected.


---
# Train-Test Split

In [10]:
window_cols = window_col_names(dataset_name)
data_array = Data[window_cols].to_numpy()

In [11]:
x_train, x_test = train_test_split(data_array, test_size=0.2, random_state=42)

---
# WandB Sweep Config
https://github.com/wandb/examples/blob/master/colabs/keras/Keras_param_opti_using_sweeps.ipynb


In [12]:
wandb.login()

wandb: Currently logged in as: vasco-mergulhao (vasco-phd). Use `wandb login --relogin` to force relogin


True

---
---
# Model & Sweep Configurations

## Generic Training Functions

In [13]:
def get_initialization (activation_fn = 'SELU'):
    # Select right initialization for respective activation function between SELU and LeakyReLU
    if activation_fn.lower() == "selu":
        return 'lecun_normal'
    if activation_fn.lower() == "leakyrelu":
        return 'he_normal'

In [14]:
def get_activation_fn (activation_fn = 'SELU'):
    # Select right initialization for respective activation function between SELU and LeakyReLU
    if activation_fn.lower() == "selu":
        return Activation('selu')
    if activation_fn.lower() == "leakyrelu":
        return LeakyReLU(alpha=0.2)

In [15]:
def get_optimizer(lr=0.0001, optimizer="nadam"):
    # Select optmizer between adam and sgd
    if optimizer.lower() == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)

In [16]:
def train(model, batch_size= 32, epochs= 100, lr=0.001, optimizer='nadam'):  
    
    tf.keras.backend.clear_session()
    model.compile(loss="mse", 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=["mse"])

    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

    model.fit(x_train, 
              x_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(x_test, x_test), 
              callbacks=[WandbCallback(), early_stopping_cb, lr_scheduler_cb])
    
    

In [17]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration
        wandb.config.architecture_name = AE_Model_Name
        wandb.config.dataset_name = dataset_name

        # initialize model
        if AE_Model_Name == 'FC_Small':
            AE_model = Small_AE(wandb.config.window_length,
                            wandb.config.latent_layer_size,
                            wandb.config.activation_fn)
        if AE_Model_Name == 'FC_Medium':
            AE_model = Medium_AE(wandb.config.window_length,
                            wandb.config.latent_layer_size,
                            wandb.config.activation_fn)

        train(AE_model, 
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.learning_rate,
              wandb.config.optimizer)
        


## FC_Small
- (Input, 200, 200, Enconder)

### Sweep Configuration

In [18]:
if AE_Model_Name == 'FC_Small':
    
    sweep_config = {
    'method': 'random',
    'name': Sweep_Config,
    }
    
    metric = {
    'name': 'mse',
    'goal': 'minimize'
    }
    sweep_config['metric'] = metric
    
    parameters_dict = {
        'optimizer': {
            'values': ['nadam', 'sgd']
        },
        'latent_layer_size': {
            'value': latent_layer_size #previous [values: [5, 10, 25]
        },
        'epochs':{
            'value': 100
        },
        'window_length':{
            'value': int(dataset_name.split('_')[-1][1:])
        },
        'activation_fn':{
            'values': ['SELU','LeakyReLU']
        }
    }
    sweep_config['parameters'] = parameters_dict
    
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 0.001,
            'max': 0.1,
          },
        'batch_size': {
            # integers between 2 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 2,
            'max': 256,
          }
        })
    print('Code Used')
else:
    print('NOT Active')

Code Used


### _Model_: FC_Small

In [19]:
def Small_AE(window_length = 90, latent_layer_size = 5, activation_fn = 'SELU'):
    
    inputs = Input(shape= window_length)
    
    layer_e1 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(inputs)
    
    layer_e2 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e1)
    #Latent Space (no activation)
    encoded = Dense(latent_layer_size)(layer_e2)
       
    layer_d1 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(encoded)
    
    layer_d2 = Dense(200, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d1)
    
    decoded = Dense(window_length)(layer_d2)
       
    autoencoder = keras.models.Model(inputs=inputs, outputs = decoded)
    
    return autoencoder   

In [20]:
if AE_Model_Name == 'FC_Small':
    AE_model = Small_AE(latent_layer_size = latent_layer_size)
    AE_model.summary()
    print('Code Used')
else:
    print('NOT Active')    

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dense (Dense)               (None, 200)               18200     
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 5)                 1005      
                                                                 
 dense_3 (Dense)             (None, 200)               1200      
                                                                 
 dense_4 (Dense)             (None, 200)               40200     
                                                                 
 dense_5 (Dense)             (None, 90)                18090 

---
## FC_Medium
-  (Input, 300, 300, 300, 300, Encoder)

### Sweep Configuration

In [21]:
if AE_Model_Name == 'FC_Medium':
    
    sweep_config = {
    'method': 'random',
    'name': Sweep_Config,
    }
    
    metric = {
    'name': 'mse',
    'goal': 'minimize'
    }
    sweep_config['metric'] = metric

    parameters_dict = {
        'optimizer': {
            'values': ['nadam', 'sgd']
        },
        'latent_layer_size': {
            'value': latent_layer_size #previous [values: [5, 10, 25]
        },
        'epochs':{
            'value': 100
        },
        'window_length':{
            'value': int(dataset_name.split('_')[-1][1:])
        },
        'activation_fn':{
            'values': ['SELU','LeakyReLU']
        }
    }
    sweep_config['parameters'] = parameters_dict
    
    parameters_dict.update({
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 0.001,
            'max': 0.1,
          },
        'batch_size': {
            # integers between 2 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 2,
            'min': 2,
            'max': 256,
          }
        })
    print('Code Used')
else:
    print('NOT Active')

NOT Active


### _Model_: FC_Medium

In [22]:
def Medium_AE(window_length = 90, latent_layer_size = 5, activation_fn = 'SELU'):
    
    inputs = Input(shape= window_length)
    
    layer_e1 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(inputs)    
    layer_e2 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e1)    
    layer_e3 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e2)    
    layer_e4 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_e3)
    #Latent Space (no activation)
    encoded = Dense(latent_layer_size)(layer_e4)
       
    layer_d1 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(encoded)
    layer_d2 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d1)
    layer_d3 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d2)
    layer_d4 = Dense(300, activation = get_activation_fn(activation_fn),
                     kernel_initializer=get_initialization(activation_fn))(layer_d3)    
    decoded = Dense(window_length)(layer_d4)
       
    autoencoder = keras.models.Model(inputs=inputs, outputs = decoded)
    
    return autoencoder   

In [23]:
if AE_Model_Name == 'FC_Medium':
    AE_model = Medium_AE(latent_layer_size = latent_layer_size)
    AE_model.summary()
    print('Code Used')
else:
    print('NOT Active')

NOT Active


---
---
# Run Sweep

In [24]:
sweep_id = wandb.sweep(sweep_config, project = Project_Name)

Create sweep with ID: ikwjt20r
Sweep URL: https://wandb.ai/vasco-phd/FC_AE-Kenya_10k_Set_1_w90/sweeps/ikwjt20r


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count= sweep_count)

wandb: Agent Starting Run: 4kpszhpo with config:
wandb: 	activation_fn: LeakyReLU
wandb: 	batch_size: 40
wandb: 	epochs: 100
wandb: 	latent_layer_size: 5
wandb: 	learning_rate: 0.025062275391764692
wandb: 	optimizer: sgd
wandb: 	window_length: 90


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
1761/1771 [============================>.] - ETA: 0s - loss: 0.6400 - mse: 0.6400

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_123503-4kpszhpo\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_123503-4kpszhpo\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_123503-4kpszhpo\files\model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_133143-73njp8t9\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_133143-73njp8t9\files\model-best)... Done. 0.0s


35419/35419 [==============================] - 99s 3ms/step - loss: 0.5268 - mse: 0.5268 - val_loss: 0.5377 - val_mse: 0.5377 - lr: 1.8803e-04
Epoch 52/100
10879/35419 [========>.....................] - ETA: 1:00 - loss: 0.5266 - mse: 0.5266

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_133143-73njp8t9\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_133143-73njp8t9\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\wandb\run-20230202_133143-73njp8t9\files\model-best)... Done. 0.0s


35419/35419 [==============================] - 93s 3ms/step - loss: 0.5250 - mse: 0.5250 - val_loss: 0.5373 - val_mse: 0.5373 - lr: 1.8803e-04
Epoch 56/100
33251/35419 [===========================>..] - ETA: 5s - loss: 0.5244 - mse: 0.5244

In [ ]:
wandb.finish()

---
# Debugging